# **Import Necessaries**

In [ ]:
#pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import warnings
warnings.filterwarnings('ignore')

eff  import  

In [ ]:
import json
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import missingno as msno

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from keras.preprocessing import text
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer

import torch
from torch.utils.data import Dataset

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
from transformers import DistilBertTokenizerFast
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from transformers import TrainingArguments, Trainer

In [ ]:

chemin_fichier_drive = '/content/data_final.xlsx'

# Charger le fichier Excel dans un DataFrame
df = pd.read_excel(chemin_fichier_drive)

# Affichez les premières lignes du DataFrame pour vérifier que tout s'est bien passé
df.head()



,Pattern,Tag,response
0,السلام عليكم هل يمكنك مساعذتي,تحية,السلام عليكم كيف يمكنني مساعدتك
1,تحية الله ممكن أسألك,تحية,مرحبا كيف يمكننني مساعدتك
2,مرحبا هل يمكنني أطرح عليك سؤال,تحية,هل هناك شيء يمكنني مساعدتك فيه
3,أهلن هل ممكن تساعدني لدي سؤال,تحية,كيف يمكنني خدمتك اليوم
4,مرحبا، هل يمكنك مساعدتي بشيء,تحية,هل يوجد طلب خاص يمكنني تلبيته


In [ ]:
labels = df['Tag'].unique().tolist()
labels = [s.strip() for s in labels]
labels

['تحية',
 'مغادرة',
 'مقر الإدارة',
 'مقرالمطعم',
 'مقرالإقامة',
 'الأقسام',
 'رئيس قطاع هندسة الكمبيوتر',
 'العميد العام',
 'تعريف',
 'التخصصات',
 'تكاليف التسجيل']

In [ ]:
num_labels = len(labels)
id2label = {id:label for id, label in enumerate(labels)}
label2id = {label:id for id, label in enumerate(labels)}

In [ ]:
id2label

{0: 'تحية',
 1: 'مغادرة',
 2: 'مقر الإدارة',
 3: 'مقرالمطعم',
 4: 'مقرالإقامة',
 5: 'الأقسام',
 6: 'رئيس قطاع هندسة الكمبيوتر',
 7: 'العميد العام',
 8: 'تعريف',
 9: 'التخصصات',
 10: 'تكاليف التسجيل'}

In [ ]:
label2id

{'تحية': 0,
 'مغادرة': 1,
 'مقر الإدارة': 2,
 'مقرالمطعم': 3,
 'مقرالإقامة': 4,
 'الأقسام': 5,
 'رئيس قطاع هندسة الكمبيوتر': 6,
 'العميد العام': 7,
 'تعريف': 8,
 'التخصصات': 9,
 'تكاليف التسجيل': 10}

In [ ]:

df['labels'] = df['Tag'].map(lambda x: label2id[x.strip()])
df.head()

,Pattern,Tag,response,labels
0,السلام عليكم هل يمكنك مساعذتي,تحية,السلام عليكم كيف يمكنني مساعدتك,0
1,تحية الله ممكن أسألك,تحية,مرحبا كيف يمكننني مساعدتك,0
2,مرحبا هل يمكنني أطرح عليك سؤال,تحية,هل هناك شيء يمكنني مساعدتك فيه,0
3,أهلن هل ممكن تساعدني لدي سؤال,تحية,كيف يمكنني خدمتك اليوم,0
4,مرحبا، هل يمكنك مساعدتي بشيء,تحية,هل يوجد طلب خاص يمكنني تلبيته,0


In [ ]:
X = list(df['Pattern'])
X[:5]

['السلام عليكم هل يمكنك مساعذتي',
 'تحية الله ممكن أسألك',
 'مرحبا هل يمكنني أطرح عليك سؤال',
 'أهلن هل ممكن تساعدني لدي سؤال',
 'مرحبا، هل يمكنك مساعدتي بشيء']

In [ ]:
y = list(df['labels'])
y[:5]

[0, 0, 0, 0, 0]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.1,random_state = 123)

In [ ]:
model_name = "bert-base-multilingual-cased"
max_len = 256

tokenizer = BertTokenizer.from_pretrained(model_name,
                                          max_length=max_len)

model = BertForSequenceClassification.from_pretrained(model_name,
                                                      num_labels=num_labels,
                                                      id2label=id2label,
                                                      label2id = label2id)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_encoding = tokenizer(X_train, truncation=True, padding=True)
test_encoding = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
full_data = tokenizer(X, truncation=True, padding=True)

In [ ]:
class DataLoader(Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):

        return len(self.labels)

In [ ]:
train_dataloader = DataLoader(train_encoding, y_train)
test_dataloader = DataLoader(test_encoding, y_test)

In [ ]:
train_dataloader[2]

{'input_ids': tensor([   101,    790,  10961,  19375,  12497,  85200,  16845,  59901, 111171,
          67151,  99295,  25705,    775,  45099,  31376,    102,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor(0)}

In [ ]:
fullDataLoader = DataLoader(full_data, y_test)

In [ ]:
def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)

    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='./output',
    do_train=True,
    do_eval=True,
    num_train_epochs=100,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.05,
    logging_strategy='steps',
    logging_dir='./multi-class-logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    load_best_model_at_end=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=test_dataloader,
    compute_metrics= compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:

q=[trainer.evaluate(eval_dataset=df2) for df2 in [train_dataloader, test_dataloader]]

pd.DataFrame(q, index=["train","test"]).iloc[:,:5]

In [ ]:
def predict(text):

    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    pred_label_idx = probs.argmax()
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [ ]:
model_path = '/content/model'
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
chatbot= pipeline("text-classification", model=model, tokenizer=tokenizer)

ajouter reponse

In [ ]:
def chat(chatbot):
  print("Chatbot: Hi! I am your virtual assistant. Feel free to ask, and I'll do my best to provide you with answers and assistance.")
  print("Type 'quit' to exit the chat\n\n")

  while True:
    text = input("User: ").strip().lower()
    if text == 'quit':
      break

    # Call your chatbot function and get the response
    response = chatbot(text)

    # Check if the chatbot has a response and confidence score is high enough
    if response and response[0]['score'] >= 0.99:  # Check for score >= 0.9
      print(f"Chatbot: {response[0]['label']}\n\n")  # Access label from response
    else:
      print("Chatbot: I don't have a response yet.\n\n")

# Call the chat function with your chatbot (replace with your actual function)
chat(chatbot)


In [ ]:

def chat(chatbot):
    print("Chatbot: Hi! I am your virtual assistant. Feel free to ask, and I'll do my best to provide you with answers and assistance.")
    print("Type 'quit' to exit the chat\n\n")

    while True:
        text = input("User: ").strip().lower()
        if text == 'quit':
            break

        # Appeler votre modèle de chatbot et obtenir la réponse
        response = chatbot(text)

        # Vérifiez si le chatbot a une réponse et si le score de confiance est suffisamment élevé
        if response and response[0]['score'] >= 0.99:  # Vérifiez si le score est >= 0.9
            tag = response[0]['label']  # Accédez à l'étiquette de la réponse
            # Recherchez la réponse associée au tag dans le DataFrame
            matched_row = df[df['Tag'] == tag]
            if not matched_row.empty:
                print(f"Chatbot: {matched_row['response'].values[0]}\n\n")
            else:
                print("Chatbot: Je n'ai pas encore de réponse.\n\n")
        else:
            print("Chatbot: Je n'ai pas encore de réponse.\n\n")

In [ ]:
chat(chatbot)